# Extract Data

## Para que serve?

Código para extrair o valor de um produto em uma determinada coordenda geográfica. Por exemplo, caso queira a quantidade de precipitação (produto das chuvas) em uma dada latitude e longitude específica. Testado apenas para alguns produtos do tipo ABI, demais casos provavelmente deverão ser feitas adaptações no código.  


## Bibliotecas necessárias:

In [ ]:
# Required modules
from datetime import datetime                   # Basic Dates and time types
import os                                       # Miscellaneous operating system interfaces
from osgeo import osr                           # Python bindings for GDAL
from osgeo import gdal                          # Python bindings for GDAL
import numpy as np                              # Scientific computing with Python
import boto3                                    # Amazon Web Services (AWS) SDK for Python
from botocore import UNSIGNED                   # boto3 config
from botocore.config import Config              # boto3 config
gdal.PushErrorHandler('CPLQuietErrorHandler')   # Ignore GDAL warnings

## Detalhando o código...

Abaixo temos a função para mudar a projeção cartográfica de acordo com a extensão desejada:

In [ ]:
def reproject(file_name, ncfile, array, extent, undef):

    # Read the original file projection and configure the output projection
    source_prj = osr.SpatialReference()
    source_prj.ImportFromProj4(ncfile.GetProjectionRef())
    target_prj = osr.SpatialReference()
    target_prj.ImportFromProj4("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
   
    # Reproject the data
    GeoT = ncfile.GetGeoTransform()
    driver = gdal.GetDriverByName('MEM')
    raw = driver.Create('raw', array.shape[0], array.shape[1], 1, gdal.GDT_Float32)
    raw.SetGeoTransform(GeoT)
    raw.GetRasterBand(1).WriteArray(array)

    # Define the parameters of the output file  
    kwargs = {'format': 'netCDF', \
            'srcSRS': source_prj, \
            'dstSRS': target_prj, \
            'outputBounds': (extent[0], extent[3], extent[2], extent[1]), \
            'outputBoundsSRS': target_prj, \
            'outputType': gdal.GDT_Float32, \
            'srcNodata': undef, \
            'dstNodata': 'nan', \
            'resampleAlg': gdal.GRA_NearestNeighbour}

    # Write the reprojected file on disk
    gdal.Warp(file_name, raw, **kwargs)

A seguir, temos a função para download do arquivo nc, faz-se uma conexão com o servidor da Amazon Web Services através da biblioteca boto3, e então baixa-se o arquivo buscando-o pelo nome e data escolhida.

In [ ]:
def download_PROD(yyyymmddhhmn, product_name, path_dest, bucket_name):

  os.makedirs(path_dest, exist_ok=True)

  year = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
  day_of_year = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%j')
  hour = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
  min = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

  # Initializes the S3 client
  s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

  # File structure
  prefix = f'{product_name}/{year}/{day_of_year}/{hour}/OR_{product_name}-M6_G16_s{year}{day_of_year}{hour}{min}'

  # Seach for the file on the server
  s3_result = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter = "/")

  # Check if there are files available
  if 'Contents' not in s3_result: 
    # There are no files
    print(f'No files found for the date: {yyyymmddhhmn}, Product-{product_name}')
    return -1
  else:
    # There are files
    for obj in s3_result['Contents']: 
      key = obj['Key']
      # Print the file name
      file_name = key.split('/')[-1].split('.')[0]

      # Download the file
      if os.path.exists(f'{path_dest}/{file_name}.nc'):
        print(f'File {path_dest}/{file_name}.nc exists')
      else:
        print(f'Downloading file {path_dest}/{file_name}.nc')
        s3_client.download_file(bucket_name, key, f'{path_dest}/{file_name}.nc')
  return f'{file_name}'

Primeiramente, define-se algumas variáveis:
- input: diretório em que ficarão os arquivos nc a serem baixados
- output: diretório em que ficarão os arquivos nc reprojetados 
- extent: recorte da extensão desejada, deve-se colocar a mínima longitude, mínima latitude, máxima longitude e máxima latitude, respectivamente. Os valores abaixo são do recorte do Brasil
- bucket_name: nome do satélite 
- product_name: nome do produto desejado, deve ser escrito exatamente como no site da Amazon (servidor utilizado)
- var: variável desejada do produto. É possível visualizar as variáveis disponíveis extraindo o conteúdo do arquivo nc, o que pode ser feito com Matlab (buscar pelo comando ncdisp)
- lon: longitude na qual se deseja o valor do produto 
- lat: latitude na qual se deseja o valor do produto 

As informações usadas nesse código são voltadas para o produto das chuvas ("ABI-L2-RRQPEF"), sendo a variável a quantidade de precipitação ('RRQPE'). Contudo, o código pode ser adaptado para outros produtos que sejam do tipo ABI, que seguem esse mesmo modelo, seria apenas necessário trocar o product_name e a var.

In [ ]:
# Input and output directories
input = "Samples"; os.makedirs(input, exist_ok=True)
output = "Output"; os.makedirs(output, exist_ok=True)

# Desired data:
extent = [-75.0, -34, -34, 5.5] # Min lon, Min lat, Max lon, Max lat (values for Brazil)
bucket_name = 'noaa-goes16'
product_name = 'ABI-L2-RRQPEF'
var = 'RRQPE'
lon = -63.15
lat = -7.603

Arquivos nc normalmente armazenam os dados em uma matriz multidimensional, no caso dos dados desse trabalho o tamanho dessa matriz tende a ser padrão (5424,5424), mas isso também pode ser consultado dentro do cabeçalho do próprio arquivo nc. Esse código baixa o arquivo mais recente disponível no servidor, que para os produtos desse trabalho (chuvas, ventos, incêndios) o intervalo é a cada dez minutos. Caso deseje uma configuração diferente, as linhas abaixo devem ser alteradas. 

In [ ]:
array = np.zeros((5424,5424))
minute = int(datetime.now().strftime('%M'))
yyyymmddhhmn = datetime.now().strftime('%Y%m%d%H' + str(minute - (minute % 10))) 

Então, o arquivo é baixado e lido, retirando valores indefinidos e aplicando a escala (dados do próprio arquivo). Além disso, também é feita a reprojeção.

In [ ]:
# Download and open the file
file_name = download_PROD(yyyymmddhhmn, product_name, input, bucket_name)
img = gdal.Open(f'NETCDF:{input}/{file_name}.nc:' + var)
dqf = gdal.Open(f'NETCDF:{input}/{file_name}.nc:DQF')

# Read the header metadata
metadata = img.GetMetadata()
scale = float(metadata.get(var + '#scale_factor'))
offset = float(metadata.get(var + '#add_offset'))
undef = float(metadata.get(var + '#_FillValue'))
dtime = metadata.get('NC_GLOBAL#time_coverage_start')
unit = metadata.get(var + '#units')

# Load the data
ds = img.ReadAsArray(0, 0, img.RasterXSize, img.RasterYSize).astype(float)
ds_dqf = dqf.ReadAsArray(0, 0, dqf.RasterXSize, dqf.RasterYSize).astype(float)

# Remove undef
ds[ds == undef] = np.nan

# Apply the scale and offset 
ds = (ds * scale + offset)

# Apply NaN's where the quality flag is greater than 1
ds[ds_dqf > 1] = np.nan

# Reproject the file
array = np.nansum(np.dstack((array, ds)),2)
filename = f'{output}/reproject_{yyyymmddhhmn}.nc'
reproject(filename, img, array, extent, undef)

Lê-se o tamanho da matriz do novo arquivo reprojetado e carrega-se esses dados.

In [ ]:
# Read number of cols and rows
sat_data = gdal.Open(filename)
ncol = sat_data.RasterXSize
nrow = sat_data.RasterYSize

# Load the data
sat_array = sat_data.ReadAsArray(0, 0, ncol, nrow).astype(float)

Utilizando a conta abaixo, é encontrado o valor do produto na coordenada desejada:

In [ ]:
# Get geotransform
transform = sat_data.GetGeoTransform()
x = int((lon - transform[0]) / transform[1])
y = int((transform[3] - lat) / -transform[5]) 
sat = sat_array[y,x]

print("value: ", sat, unit)

Cabe ressaltar que o código foi feito com a ajuda do curso “Processamento de Dados de Satélites Geoestacionários com Python” fornecido pelo INPE (Instituto Nacional de Pesquisas Espaciais).